In [15]:
import xlrd
book = xlrd.open_workbook("data/Table S2 - Positional variability and color of all neurons in males and hermaphrodites v2.xlsx")


print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))

print("Worksheet names: {0}".format(book.sheet_names()))

data = {}

for ws in range(book.nsheets):
    sheet_name = book.sheet_names()[ws]
    data[sheet_name]={}
    
    print('Parsing sheet: %s'%sheet_name)
    sh = book.sheet_by_index(ws)
    
    for rx in range(sh.nrows):
        if rx==1:
            print('  Row %i: %s'%(rx,sh.row(rx)))
            
        if rx==1:
            neuron = sh.row(rx)[0]
            data[sheet_name][neuron]={}
            data[sheet_name][neuron]['volume'] = sh.row(rx)[1].value
            data[sheet_name][neuron]['ap_position'] = sh.row(rx)[2].value
            data[sheet_name][neuron]['dv_position'] = sh.row(rx)[3].value
            data[sheet_name][neuron]['lr_position'] = sh.row(rx)[4].value
            #'A-P Distance (μm)	D-V Distance (μm)	L-R Distance (μm)	A-P Variance (μm2)	D-V Variance (μm2)	L-R Variance (μm2)	Effect Size (D)
        
            
print(data)
            
            

Male Tail Colors 111 7
Worksheet names: ['Hermaphrodite Head Positions', 'Hermaphrodite Head Colors', 'Hermaphrodite Tail Positions', 'Hermaphrodite Tail Colors', 'Male Head Positions', 'Male Head Colors', 'Male Tail Positions', 'Male Tail Colors']
Parsing sheet: Hermaphrodite Head Positions
  Row 1: [text:'ADAL', number:182.2682051580339, number:100.8276428579064, number:23.60987581238903, number:10.56640787942424, number:17.6163994526686, number:6.531587864448621, number:5.967212931466882, number:30.50263805730735, number:3.045720378507932, number:3.197722178523348, number:-0.2066570664170569]
Parsing sheet: Hermaphrodite Head Colors
  Row 1: [text:'ADAL', number:0.4168104770064675, number:0.01080264309343614, number:0.140685171108576, number:0.001569131409321875, number:0.766831704491221, number:0.05966809666998414]
Parsing sheet: Hermaphrodite Tail Positions
  Row 1: [text:'ALNL', number:9.904192141814923, number:32.33366272020893, number:20.29171729401544, number:21.49260023496281